In [2]:
import warnings
warnings.filterwarnings("ignore")
import os
import json
import pandas as pd
import numpy as np
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.efficientnet import preprocess_input, EfficientNetB3
from keras.optimizers import Adam
from keras import layers, Model
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns
import multiprocessing

In [3]:
# Use CPU/GPU strategy
strategy = tf.distribute.get_strategy()
print("REPLICAS: ", strategy.num_replicas_in_sync)

REPLICAS:  1


In [4]:
# Hyperparameters
batch_size = 32 * strategy.num_replicas_in_sync
img_size = 299
epochs = 100
workers = multiprocessing.cpu_count()

In [4]:
model = models.Sequential([
    layers.Input(shape=(150, 150, 3)),  # Ensure input size is correctly matched
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.GlobalAveragePooling2D(),  # Automatically handles different input sizes
    layers.Dense(128, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 74, 74, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 72, 72, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 36, 36, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 34, 34, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 17, 17, 128)      0

In [5]:
# Compile Model
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Callbacks
early_stopping = EarlyStopping(
    monitor='val_accuracy',
    patience=5,
    restore_best_weights=True,
    verbose=1
)

model_checkpoint = ModelCheckpoint(
    "best_food_model.keras",
    save_best_only=True,
    monitor='val_accuracy',
    mode='max',
    verbose=1
)

class StopTrainingAtAccuracy(tf.keras.callbacks.Callback):
    def __init__(self, accuracy_threshold=0.81):
        super(StopTrainingAtAccuracy, self).__init__()
        self.accuracy_threshold = accuracy_threshold

    def on_epoch_end(self, epoch, logs=None):
        if logs.get('accuracy') >= self.accuracy_threshold:
            print(f"\nStopping training as accuracy reached {self.accuracy_threshold*100:.2f}%")
            self.model.stop_training = True

stop_training = StopTrainingAtAccuracy(accuracy_threshold=0.81)

# Train the model
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=5,
    callbacks=[early_stopping, model_checkpoint, stop_training],
    verbose=1,
)

Epoch 1/5
 566/1249 [============>.................] - ETA: 5:22 - loss: 0.7110 - accuracy: 6.0733e-04


KeyboardInterrupt

